<a href="https://colab.research.google.com/github/Carmen-N/An-lisis-del-mercado-de-los-videojuegos/blob/master/video_games_sales.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Análisis del mercado de los videojuegos**


Objetivo:

Para una tienda online de juegos de ordenador, es necesario realizar un análisis de mercado, identificar los perfiles que determinan el éxito de los juegos, definir un retrato de usuario para cada una de las regiones y poner a prueba las hipótesis de trabajo. Esto permitirá apostar por un producto potencialmente popular y planificar campañas publicitarias que ayuden a desarrollar el negocio de la tienda.

Los datos se recogen de fuentes abiertas y se guardan en el archivo games.csv. Los datos incluyen información de 1983 a 2016. Es posible que los datos de 2016 no estén completos.

Descripción de los datos games.csv:

Nombre - nombre del juego

* Plataforma 



* Year_of_Release - año de publicación

* Género - género del juego


* NA_sales - Ventas en Norteamérica (millones de dólares)
* EU_sales - Ventas en Europa (millones de dólares


* JP_sales - Ventas en Japón (millones de dólares)

* Otras_ventas - ventas en otros países (millones de dólares)

* Critic_Score - puntuación de los críticos (de 0 a 100)
* Puntuación_usuario - valoración del usuario (de 0 a 10)  


* Clasificación - Clasificación proporcionada por la ESRB (Entertainment Software Rating Board). Esta asociación determina la clasificación de los juegos de ordenador y los asigna a la categoría de edad adecuada.  







**Las etapas de la ejecución de un proyecto**

1. Obtener los  datos
2. Preprocesamiento de datos
3. Análisis exploratorio de datos
4. Perfil de usuario de cada región
5. Encuesta sobre indicadores estadísticos
6. Comprobación de hipótesis
7. Resultados del estudio

Paso 1: Obtener los datos

Examine los datos aportados para el proyecto.

In [5]:
#importar librerias 
import pandas as pd
import numpy as np
from scipy import stats as st
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
import pylab as plt
sns.set(style="white")
from google.colab import files
import io 

Leer el archivo games.csv y almacenarlo en la variable df.

In [6]:
data = files.upload()

Saving Video Games Dataset.csv to Video Games Dataset.csv


In [7]:
df = pd.read_csv(io.StringIO(data['Video Games Dataset.csv'].decode('utf-8')))

In [9]:

# Obtener las 10 primeras filas de la tabla.
df.head(10)

,Rank,Name,Platform,Year,Genre,Publisher,NorthAmerica_Sales,EurpeanUnion_Sales,Japan_Sales,Other_Sales,Global_Sales
0,1,Wii Sports,Wii,2006,Sports,Nintendo,41.49,29.02,3.77,8.46,82.74
1,2,Super Mario Bros.,NES,1985,Platform,Nintendo,29.08,3.58,6.81,0.77,40.24
2,3,Mario Kart Wii,Wii,2008,Racing,Nintendo,NaN,12.88,3.79,NaN,35.82
3,4,Wii Sports Resort,Wii,2009,Sports,Nintendo,15.75,11.01,NaN,2.96,33.00
4,5,Pokemon Red/Pokemon Blue,GB,1996,Role-Playing,Nintendo,11.27,8.89,10.22,1.00,31.37
5,6,Tetris,GB,1989,Puzzle,Nintendo,23.20,2.26,4.22,0.58,NaN
6,7,New Super Mario Bros.,DS,2006,Platform,Nintendo,11.38,9.23,6.50,2.90,30.01
7,8,Wii Play,Wii,2006,Misc,Nintendo,14.03,9.20,2.93,2.85,29.02
8,9,New Super Mario Bros. Wii,Wii,2009,Platform,Nintendo,14.59,7.06,NaN,2.26,28.62
9,10,Duck Hunt,NES,1984,Shooter,Nintendo,26.93,NaN,0.28,0.47,28.31


Información general sobre los datos de la tabla df.

In [10]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 199 entries, 0 to 198
Data columns (total 11 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   Rank                199 non-null    int64  
 1   Name                199 non-null    object 
 2   Platform            199 non-null    object 
 3   Year                199 non-null    int64  
 4   Genre               199 non-null    object 
 5   Publisher           198 non-null    object 
 6   NorthAmerica_Sales  185 non-null    float64
 7   EurpeanUnion_Sales  190 non-null    float64
 8   Japan_Sales         189 non-null    float64
 9   Other_Sales         192 non-null    float64
 10  Global_Sales        187 non-null    float64
dtypes: float64(5), int64(2), object(4)
memory usage: 17.2+ KB


Veamos más de cerca la información obtenida.

Hay un total de 11 columnas en la tabla. Hay dos tipos de datos: objeto no nulo (5 columnas) y float64 no nulo (6 columnas).

Resultados

Cada fila de la tabla contiene datos históricos sobre ventas de juegos, valoraciones de usuarios y expertos, géneros y plataformas (por ejemplo, Xbox o PlayStation), año de lanzamiento y clasificación por edades asignada.

Dos problemas que hay que resolver en la próxima fase: las omisiones y los nombres poco acertados de las columnas.

Las columnas Género, Plataforma, Año_de_estreno y desglose de ventas por regiones son especialmente valiosas para comprobar hipótesis de trabajo. Podemos utilizarlos para rastrear cómo han cambiado las preferencias de los usuarios a lo largo del tiempo, para construir un perfil de usuario para cada región. Las columnas Critic_Score, User_Score, Rating nos ayudarán a probar hipótesis sobre el efecto de las puntuaciones de los juegos en las ventas en función del género y la plataforma del juego.

Paso 2: Procesamiento previo de los datos

Cambie los nombres de las columnas, convierta los datos al tipo correcto, procese las lagunas (rellenándolas cuando sea posible), compruebe si hay datos duplicados y calcule las ventas totales en todas las regiones y anótelas en una columna aparte.

Obtenemos una lista de nombres de columnas.

In [11]:
df.columns

Index(['Rank', 'Name', 'Platform', 'Year', 'Genre', 'Publisher',
       'NorthAmerica_Sales', 'EurpeanUnion_Sales', 'Japan_Sales',
       'Other_Sales', 'Global_Sales'],
      dtype='object')

Los nombres de las columnas están en mayúsculas, lo que no resulta muy cómodo para acceder a los datos. Vamos a cambiar el nombre de las columnas para facilitar la consulta. 
Comprobemos el resultado.

In [12]:
df = df.rename(columns=lambda x: x.lower())

In [13]:
df.columns

Index(['rank', 'name', 'platform', 'year', 'genre', 'publisher',
       'northamerica_sales', 'eurpeanunion_sales', 'japan_sales',
       'other_sales', 'global_sales'],
      dtype='object')

Es necesario determinar si hay duplicados. 
Si los encuentra, elimínelos y compruebe que se han eliminado todos.

In [15]:
# obtener el número total de duplicados en la tabla df
df.duplicated() .sum()

0

No se han encontrado duplicados. Tratemos los valores que faltan.

Comprobar si faltan valores en los datos llamando a un conjunto de métodos para sumar los valores que faltan.

In [16]:
df.isnull() .sum()

rank                   0
name                   0
platform               0
year                   0
genre                  0
publisher              1
northamerica_sales    14
eurpeanunion_sales     9
japan_sales           10
other_sales            7
global_sales          12
dtype: int64

Tratamiento de las omisiones en la columna "nombre

En primer lugar, veamos las filas con lagunas de datos en la columna del nombre.